<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0df1f4f035bb9288fe666b068983b882293ba6a1b1c22b297bb3b66f58029bb4
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-27 11:20:19
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.44 C
-------------------
Today PnL: -7.50 K (-0.05%)
Current PnL: -22.19 L (-14.54%)
CY Booked + Current PnL: -8.30 L (-5.44%)
-------------------
Total profit:  1.74 L
Total loss:  -23.93 L
-------------------
Total Booked + Current PnL: 18.68 L (14.87%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.67%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.60 L (61.0%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 7.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,-0.05,-14.16,16.60,0.09,14361.0,-14265.0,86512.0,100.85,50.0,M-SC,3.54,253.0,-0.99,0.60,13.66,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,-3.18,-10.14,11.30,0.02,17726.0,-17693.0,156866.0,139.40,63.0,M-SC,9.08,234.0,-1.00,1.09,36.45,OX40N,NTT,PAINTS
53,NESTLEIND,1377.0,-0.05,12.51,7.90,21.40,23592.0,33205.0,298631.0,4.42,58.0,X-LC,7.68,12.0,1.41,2.08,19.98,XY25,NTT,FMCG
83,VOLTAS,1530.0,-1.07,7.35,11.50,19.69,23670.0,14088.0,205830.0,-3.15,49.0,X-MC,3.33,78.0,0.60,1.43,14.64,XY25,NTT,AC
43,ITC,452.0,0.19,-1.52,12.14,10.44,23927.0,-3047.0,197091.0,-41.15,44.0,X-LC,1.02,5.0,-0.13,1.37,3.96,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,4.95,4.51,26.70,32.42,70848.0,11454.0,265350.0,-14.09,54.0,X-SC,7.50,85.0,0.16,1.85,20.29,X40,ATH,FMCG
58,RAJOOENG,143.10,4.57,-24.05,83.82,39.61,65254.0,-24650.0,77850.0,-48.54,42.0,H-SC,20.44,135.0,-0.38,0.54,9.73,AR,ATH,MISC
67,SIS,528.00,2.37,-21.04,53.58,21.26,46973.0,-23363.0,87669.0,2072.54,60.0,H-SC,3.79,166.0,-0.50,0.61,18.45,OX40N,NTT,MISC
68,SONACOMS,806.63,1.81,-10.67,56.20,39.54,50788.0,-10789.0,90370.0,-30.10,70.0,M-SC,6.64,220.0,-0.21,0.63,27.43,AR,ATH,AUTO
21,DEN,75.00,1.75,-39.34,134.23,42.07,68769.0,-33232.0,51232.0,81.35,47.0,M-SC,8.16,235.0,-0.48,0.36,8.80,AR,NTT,ENTERTAINMENT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-10.90,-14.25,112.15,81.92,110400.0,-16358.0,98440.0,-49.73,23.0,M-SC,3.33,236.0,-0.15,0.69,16.54,XR,NTT,DURABLES
39,INDIGOPNTS,1408.0,-3.18,-10.14,11.30,0.02,17726.0,-17693.0,156866.0,139.40,63.0,M-SC,9.08,234.0,-1.00,1.09,36.45,OX40N,NTT,PAINTS
70,SURYODAY,216.0,-2.02,-19.91,53.63,23.03,76912.0,-35659.0,143412.0,55.70,48.0,H-SC,9.45,167.0,-0.46,1.00,42.29,XR,NTT,BANKS
28,GREENPANEL,537.0,-1.88,-32.46,111.92,43.14,138399.0,-59419.0,123659.0,177.13,32.0,M-SC,5.85,240.0,-0.43,0.86,16.32,XY24,NTT,MISC
14,CAMPUS,393.0,-1.66,-8.35,45.42,33.28,69843.0,-14003.0,153772.0,-26.15,47.0,M-SC,2.97,222.0,-0.20,1.07,19.18,XY24,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.0,0.54,105.62,106.72,130968.0,663.0,123999.0,-50.25,39.0,H-SC,3.09,139.0,0.01,0.86,24.04,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.26,-4.45,117.60,107.91,165902.0,-6571.0,141073.0,-22.17,37.0,M-SC,10.56,216.0,-0.04,0.98,0.88,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-3.18,-10.14,11.30,0.02,17726.0,-17693.0,156866.0,139.40,63.0,M-SC,9.08,234.0,-1.00,1.09,36.45,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.05,-14.16,16.60,0.09,14361.0,-14265.0,86512.0,100.85,50.0,M-SC,3.54,253.0,-0.99,0.60,13.66,OX40N,NTT,DURABLES
67,SIS,528.00,2.37,-21.04,53.58,21.26,46973.0,-23363.0,87669.0,2072.54,60.0,H-SC,3.79,166.0,-0.50,0.61,18.45,OX40N,NTT,MISC
47,KANSAINER,340.00,0.41,-21.58,44.69,13.47,94511.0,-58185.0,211482.0,-67.73,35.0,H-SC,1.49,158.0,-0.62,1.47,7.53,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.15,2.47,68.12,72.27,118273.0,4177.0,173625.0,-11.98,60.0,M-LC,3.86,99.0,0.04,1.21,8.79,XR,NTT,IT
38,INDIAMART,4810.62,0.00,0.54,105.62,106.72,130968.0,663.0,123999.0,-50.25,39.0,H-SC,3.09,139.0,0.01,0.86,24.04,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.15,2.47,68.12,72.27,118273.0,4177.0,173625.0,-11.98,60.0,M-LC,3.86,99.0,0.04,1.21,8.79,XR,NTT,IT
38,INDIAMART,4810.62,0.00,0.54,105.62,106.72,130968.0,663.0,123999.0,-50.25,39.0,H-SC,3.09,139.0,0.01,0.86,24.04,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,-0.23,-1.05,37.07,35.63,76874.0,-2199.0,207375.0,-14.97,42.0,H-MC,3.97,119.0,-0.03,1.44,15.45,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.26,-4.45,117.60,107.91,165902.0,-6571.0,141073.0,-22.17,37.0,M-SC,10.56,216.0,-0.04,0.98,0.88,OX40N,ATH,CABLES
37,IEX,219.00,-0.42,-4.19,55.13,48.63,105611.0,-8386.0,191568.0,-36.24,53.0,H-SC,15.61,136.0,-0.08,1.33,6.87,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.39,-35.22,108.72,35.20,90617.0,-45321.0,83349.0,9.31,27.0,X-SC,13.18,92.0,-0.50,0.58,0.39,X40,NTT,FOOTWEAR
76,TMPV,600.00,0.40,-14.53,66.34,42.18,156734.0,-40152.0,236258.0,-23.08,30.0,X-LC,1.45,3.0,-0.26,1.65,2.34,XY24,NTT,AUTO
27,GLAXO,3466.20,-0.69,-0.76,40.21,39.15,79526.0,-1508.0,197776.0,-20.38,32.0,X-MC,10.10,60.0,-0.02,1.38,28.03,X40,ATH,PHARMA
55,PGHH,17907.65,0.37,-4.22,39.65,33.76,76265.0,-8475.0,192345.0,-32.38,35.0,X-MC,3.71,57.0,-0.11,1.34,0.76,X40,ATH,FMCG
46,JSWINFRA,342.00,1.48,-7.77,24.93,15.23,46066.0,-15566.0,184781.0,-21.02,36.0,X-MC,6.88,66.0,-0.34,1.29,21.98,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.19,-1.52,12.14,10.44,23927.0,-3047.0,197091.0,-41.15,44.0,X-LC,1.02,5.0,-0.13,1.37,3.96,X40,NTT,FMCG
76,TMPV,600.0,0.40,-14.53,66.34,42.18,156734.0,-40152.0,236258.0,-23.08,30.0,X-LC,1.45,3.0,-0.26,1.65,2.34,XY24,NTT,AUTO
3,ACC,3906.0,-0.67,-21.33,108.66,64.15,203401.0,-50761.0,187190.0,-54.13,54.0,X-SC,1.48,82.0,-0.25,1.30,5.00,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,0.63,3.75,26.24,30.97,48113.0,6621.0,183359.0,-20.69,59.0,X-MC,1.83,75.0,0.14,1.28,16.78,X40,ATH,INSURANCE
66,SIEMENS,4671.5,-1.05,-11.76,42.25,25.51,69377.0,-21890.0,164205.0,0.72,62.0,H-LC,1.87,49.0,-0.32,1.14,21.01,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.39,-35.22,108.72,35.20,90617.0,-45321.0,83349.0,9.31,27.0,X-SC,13.18,92.0,-0.50,0.58,0.39,X40,NTT,FOOTWEAR
55,PGHH,17907.65,0.37,-4.22,39.65,33.76,76265.0,-8475.0,192345.0,-32.38,35.0,X-MC,3.71,57.0,-0.11,1.34,0.76,X40,ATH,FMCG
54,PAGEIND,51605.08,-0.37,-5.08,32.81,26.07,50993.0,-8320.0,155420.0,-30.06,38.0,X-MC,9.37,55.0,-0.16,1.08,0.83,X40,ATH,APPARELS
9,BAJAJHFL,181.50,1.03,-13.23,71.10,48.45,123693.0,-26535.0,173971.0,-19.44,38.0,X-MC,7.28,64.0,-0.21,1.21,1.07,X40N,ATH,FINANCE
19,COLPAL,3726.84,-0.34,-17.32,71.15,41.51,154929.0,-45615.0,217750.0,-1.46,46.0,X-MC,7.16,61.0,-0.29,1.52,2.18,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,0.33,-27.37,96.66,42.84,45639.0,-17790.0,47216.0,-52.62,42.0,X-SC,37.54,83.0,-0.39,0.33,2.74,XY24,NTT,MISC
60,RELAXO,1176.00,-0.67,-46.95,187.07,52.31,144070.0,-68146.0,77014.0,-43.47,38.0,X-SC,4.44,91.0,-0.47,0.54,3.09,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.39,-35.22,108.72,35.20,90617.0,-45321.0,83349.0,9.31,27.0,X-SC,13.18,92.0,-0.50,0.58,0.39,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.92,-5.66,28.88,21.58,38180.0,-7938.0,132201.0,-52.41,37.0,X-MC,6.57,56.0,-0.21,0.92,17.04,X40N,ATH,IT
52,MEDANTA,1486.00,-0.66,4.79,18.41,24.08,24491.0,6080.0,133030.0,-3.72,49.0,X-SC,5.54,89.0,0.25,0.93,25.34,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.29,-14.31,39.20,19.28,116208.0,-49508.0,296448.0,-25.24,61.0,X-LC,7.23,1.0,-0.43,2.06,9.59,X40,ATH,IT
41,INFY,2275.00,0.39,6.47,45.46,54.87,151442.0,20244.0,333132.0,-17.09,62.0,X-LC,2.62,2.0,0.13,2.32,15.29,X40,BTT,IT
76,TMPV,600.00,0.40,-14.53,66.34,42.18,156734.0,-40152.0,236258.0,-23.08,30.0,X-LC,1.45,3.0,-0.26,1.65,2.34,XY24,NTT,AUTO
82,VBL,671.64,0.11,-5.87,44.13,35.67,131202.0,-18534.0,297308.0,-17.02,54.0,X-LC,5.77,4.0,-0.14,2.07,7.17,X40N,ATH,FMCG
43,ITC,452.00,0.19,-1.52,12.14,10.44,23927.0,-3047.0,197091.0,-41.15,44.0,X-LC,1.02,5.0,-0.13,1.37,3.96,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.20,-37.07,111.17,32.89,54014.0,-28620.0,48587.0,-699.86,64.0,L-MC,5.63,259.0,-0.53,0.34,33.80,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.76,-13.31,109.58,81.67,89110.0,-12490.0,81320.0,7311.11,50.0,L-SC,19.27,271.0,-0.14,0.57,57.48,XR,NTT,CERAMICS
68,SONACOMS,806.63,1.81,-10.67,56.20,39.54,50788.0,-10789.0,90370.0,-30.10,70.0,M-SC,6.64,220.0,-0.21,0.63,27.43,AR,ATH,AUTO
51,MASFIN,398.61,-0.50,-6.32,30.29,22.05,27802.0,-6195.0,91785.0,-19.15,46.0,H-SC,6.96,164.0,-0.22,0.64,33.42,XR,ATH,FINANCE
70,SURYODAY,216.00,-2.02,-19.91,53.63,23.03,76912.0,-35659.0,143412.0,55.70,48.0,H-SC,9.45,167.0,-0.46,1.00,42.29,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-3.18,-10.14,11.30,0.02,17726.0,-17693.0,156866.0,139.40,63.0,M-SC,9.08,234.0,-1.00,1.09,36.45,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.64,1.10,19.63,20.95,49964.0,2763.0,254531.0,0.07,77.0,X-LC,14.57,31.0,0.06,1.77,37.12,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.20,-37.07,111.17,32.89,54014.0,-28620.0,48587.0,-699.86,64.0,L-MC,5.63,259.0,-0.53,0.34,33.80,XR,NTT,BANKS
0,5PAISA,593.00,-0.49,-34.86,73.32,12.90,104359.0,-76162.0,142334.0,109.13,58.0,H-SC,13.40,161.0,-0.73,0.99,18.35,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,-0.55,6.62,13.27,20.77,33127.0,15497.0,249641.0,-2.63,63.0,X-MC,3.77,62.0,0.47,1.74,13.99,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.85
1,25,44.74
2,50,75.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.57
MC    29.21
LC    25.23
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.85
X40      21.22
X40N     11.97
XR        9.53
XY25      9.10
AR        9.08
OX40N     7.65
X200      1.86
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.88
X-MC    22.73
X-LC    20.05
M-SC    11.95
X-SC     8.33
H-MC     4.75
H-LC     3.00
L-SC     1.41
M-MC     1.39
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.85,-4.07,37.73
IT,13.76,-14.35,73.39
FINANCE,9.44,-14.34,62.66
MISC,7.43,-21.62,71.45
ELECTRICAL,5.97,-9.40,49.47
PAINTS,5.95,-7.92,23.95
INSURANCE,4.42,0.36,34.75
PHARMA,3.85,-0.99,33.31
AUTO,3.39,-18.36,69.11


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3036897.0,21
XR,1296192.0,13
AR,1266058.0,10
X40,990303.0,14
X40N,731223.0,9
OX40N,693167.0,10
XY25,334790.0,6
SR,279902.0,2
MH,73797.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3547093.0,25
M-SC,1343179.0,15
X-MC,1219481.0,16
X-LC,883073.0,11
X-SC,725194.0,8
H-MC,396275.0,3
L-SC,260361.0,3
H-LC,126900.0,2
M-LC,118273.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1215802.0      6
           AR         868948.0      5
           XR         792170.0      7
M-SC       XY24       775555.0      6
X-MC       X40        440833.0      7
X-LC       X40        388005.0      5
X-MC       XY24       358983.0      3
H-SC       OX40N      316474.0      4
X-SC       X40N       290198.0      3
M-SC       OX40N      289059.0      5
H-SC       SR         279902.0      2
X-SC       XY24       273531.0      3
X-MC       X40N       241066.0      4
X-LC       XY24       224927.0      2
H-MC       AR         208176.0      2
X-LC       X40N       199959.0      2
H-MC       XY24       188099.0      1
X-MC       XY25       178599.0      2
L-SC       XR         172727.0      2
X-SC       X40        161465.0      2
M-SC       XR         159008.0      2
           AR         119557.0      2
M-LC       XR         118273.0      1
L-SC       OX40N       87634.0      1
H-SC       MH          73797.0      1
X-LC       XY25        70182.0      2
H-LC       AR          69377.0      1
           X200        57523.0      1
L-MC       XR          54014.0      1
M-MC       XY25        46191.0      1
L-LC       XY25        39818.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
